# Gravitee.io setup

## Env Init and MongoDB setup

In [46]:
export KUBECONFIG=`pwd`/kubeconfig.yaml
export PATH=`pwd`:$PATH

First we'll create a secret for our MongoDb database

In [47]:
cat > config.yml <<EOF
apiVersion: v1
kind: Secret
metadata:
  name: gravitee-mongodb
type: Opaque
data:
  database: gravitee
stringData:
  username: gravitee
  password: "Sup3rStr0ngPassw0rd"
EOF

In [48]:
kubectl apply -f ./config.yml

secret/gravitee-mongodb configured


We need now to create a user account associated with the MongoDB database we'll be using

In [50]:
MONGODB_ADMIN_USER=root
MONGODB_ADMIN_PASS=$(kubectl get secret my-mongodb -o jsonpath="{.data.mongodb-root-password}" | base64 --decode)
MONGODB_NS=$(kubectl get service -l release=my-mongodb -o jsonpath="{.items[0].metadata.namespace}")
MONGODB_HOST=$(kubectl get service -l release=my-mongodb -o jsonpath="{.items[0].metadata.name}")
MONGODB_PORT=$(kubectl get service -l release=my-mongodb -o jsonpath="{.items[0].spec.ports[0].port}")
MONGODB_DB=$(kubectl get secret gravitee-mongodb -o jsonpath="{.data.database}")
MONGODB_GRAVITEE_USER=$(kubectl get secret gravitee-mongodb -o jsonpath="{.data.username}" | base64 --decode)
MONGODB_GRAVITEE_PASS=$(kubectl get secret gravitee-mongodb -o jsonpath="{.data.password}" | base64 --decode)

In [51]:
# https://docs.mongodb.com/manual/reference/built-in-roles/#database-user-roles
cat > config.js <<EOF
db = db.getSiblingDB("$MONGODB_DB")
db.createUser({"user": "$MONGODB_GRAVITEE_USER", "pwd": "$MONGODB_GRAVITEE_PASS", "roles": ["dbOwner"]})
EOF

In [35]:
cat config.js | kubectl run --namespace $MONGODB_NS my-mongodb-client --rm -i --restart='Never' --image bitnami/mongodb --command -- mongo --host $MONGODB_HOST:$MONGODB_PORT -u root -p $MONGODB_PASSWORD

If you don't see a command prompt, try pressing enter.
gravitee
Successfully added user: { "user" : "gravitee", "roles" : [ "dbOwner" ] }
bye
2019-03-13T21:31:50.665+0000 E -        [main] Error saving history file: FileOpenFailed: Unable to open() file //.dbshell: Unknown error
pod "my-mongodb-client" deleted


## Gravitee installation

The first step is to build the helm [chart](https://github.com/gravitee-io/gravitee-kubernetes/tree/master/gravitee).

In [52]:
mkdir build && cd build
curl -OL https://github.com/gravitee-io/gravitee-kubernetes/archive/master.zip

mkdir: cannot create directory ‘build’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   136    0   136    0     0    279      0 --:--:-- --:--:-- --:--:--   280
100 26238    0 26238    0     0  26529      0 --:--:-- --:--:-- --:--:-- 57039


In [53]:
unzip master.zip

Archive:  master.zip
1d3ae9520d30d9e9257ef13798045e9c740e3311
   creating: gravitee-kubernetes-master/
  inflating: gravitee-kubernetes-master/.gitignore  
  inflating: gravitee-kubernetes-master/CONTRIBUTING.adoc  
  inflating: gravitee-kubernetes-master/LICENSE.txt  
  inflating: gravitee-kubernetes-master/README.adoc  
   creating: gravitee-kubernetes-master/gravitee/
  inflating: gravitee-kubernetes-master/gravitee/.helmignore  
  inflating: gravitee-kubernetes-master/gravitee/Chart.yaml  
  inflating: gravitee-kubernetes-master/gravitee/NOTES.txt  
  inflating: gravitee-kubernetes-master/gravitee/README.md  
   creating: gravitee-kubernetes-master/gravitee/templates/
  inflating: gravitee-kubernetes-master/gravitee/templates/_helpers.tpl  
  inflating: gravitee-kubernetes-master/gravitee/templates/api-autoscaler.yaml  
  inflating: gravitee-kubernetes-master/gravitee/templates/api-configmap.yaml  
  inflating: gravitee-kubernetes-master/gravitee/templates/api-deployment.yaml  
  i

In [54]:
cd gravitee-kubernetes-master/gravitee/

In [55]:
helm package .

Successfully packaged chart and saved it to: /home/pierre/workspace/articles/k8s_201/gravitee-kubernetes-master/gravitee/gravitee-1.22.0.tgz


In [56]:
MONGODB_RS=$(kubectl get replicaset -l release=my-mongodb -o jsonpath="{.items[0].metadata.name}")
MONGODB_HOST=$(kubectl get service -l release=my-mongodb -o jsonpath="{.items[0].metadata.name}.{.items[0].metadata.namespace}.svc.cluster.local")

In [57]:
echo Replicat: $MONGODB_RS, Host: $MONGODB_HOST, Port: $MONGODB_PORT

Replicat: my-mongodb-dc4f8dd66, Host: my-mongodb.default.svc.cluster.local, Port: 27017


In [58]:
echo $MONGODB_GRAVITEE_USER:$MONGODB_GRAVITEE_PASS@$MONGODB_HOST

gravitee:Sup3rStr0ngPassw0rd@my-mongodb.default.svc.cluster.local


In [59]:
ES_PROTOCOL=$(kubectl get service -l release=my-elasticsearch -o jsonpath="{.items[0].spec.ports[0].name}")
ES_PORT=$(kubectl get service -l release=my-elasticsearch -o jsonpath="{.items[0].spec.ports[0].port}")
ES_HOST=$(kubectl get service -l release=my-elasticsearch -o jsonpath="{.items[0].metadata.name}.{.items[0].metadata.namespace}.svc.cluster.local")

In [60]:
echo Protocol: $ES_PROTOCOL, Host: $ES_HOST, Port: $ES_PORT

Protocol: http, Host: elasticsearch-master-headless.default.svc.cluster.local, Port: 9200


In [61]:
helm del --purge gravitee
helm install --name gravitee gravitee-1.22.0.tgz \
--set mongo.rs=$MONGODB_RS \
--set mongo.rsEnabled=false \
--set mongo.dbhost=$MONGODB_GRAVITEE_USER:$MONGODB_GRAVITEE_PASS@$MONGODB_HOST \
--set mongo.dbport=$MONGODB_PORT \
--set es.protocol=$ES_PROTOCOL \
--set es.host=$ES_HOST \
--set es.port=$ES_PORT


Error: release: "gravitee" not found
NAME:   gravitee
LAST DEPLOYED: Wed Mar 13 22:37:06 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME              DATA  AGE
gravitee-api      1     1s
gravitee-gateway  1     1s
gravitee-ui       4     1s

==> v1/Pod(related)
NAME                               READY  STATUS             RESTARTS  AGE
gravitee-api-7ccb5794b5-vh82v      0/1    ContainerCreating  0         1s
gravitee-gateway-864d77d57d-h9sf2  0/1    ContainerCreating  0         1s
gravitee-gateway-864d77d57d-tn9nl  0/1    ContainerCreating  0         1s
gravitee-ui-5cff4d56f5-2qs4z       0/1    ContainerCreating  0         1s

==> v1/Service
NAME              TYPE       CLUSTER-IP    EXTERNAL-IP  PORT(S)   AGE
gravitee-api      ClusterIP  10.3.27.225   <none>       83/TCP    1s
gravitee-gateway  ClusterIP  10.3.243.228  <none>       82/TCP    1s
gravitee-ui       ClusterIP  10.3.215.136  <none>       8002/TCP  1s

==> v1beta1/Deployment
NAME              READY

Check that the pods are up and running

In [62]:
kubectl get pods -l release=gravitee

NAME                                READY   STATUS    RESTARTS   AGE
gravitee-api-7ccb5794b5-vh82v       0/1     Running   0          8s
gravitee-gateway-864d77d57d-h9sf2   0/1     Running   0          8s
gravitee-gateway-864d77d57d-tn9nl   0/1     Running   0          8s
gravitee-ui-5cff4d56f5-2qs4z        0/1     Running   0          8s
